In [9]:
from scipy.stats import chi2_contingency
import pandas as pd

import polars as pl
from functions.feat_network import get_edge_node_table

import sys
sys.path.append('../')

path = 'global_before_1700'

df = pd.read_csv(f"../networks/data/{path}.csv",index_col=[0])
df["meta_occupation"] = df["meta_occupation"].apply(lambda x: x.split(" | "))
df = df.explode("meta_occupation")
df = df[["wikidata_id", "meta_occupation"]]
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df.columns = ["source", "target"]
df["weight"] = 1

## get co-occurence table
res = pl.from_pandas(df)
df_edge, df_nodes = get_edge_node_table(res)
df_pivot = df_edge.pivot(index='source', columns='target', values='weight').fillna(0)

# Perform the chi-squared test
chi2, p, _, _ = chi2_contingency(df_pivot)

# Output the result
print(f"Chi-squared = {chi2}, p-value = {p}")
if p < 0.001:
    print("The associations between scholars' occupations are not random.")
else:
    print("The associations between scholars' occupations may be random.")


Chi-squared = 5627.198728955837, p-value = 0.0
The associations between scholars' occupations are not random.
